In [133]:
import numpy as np
import matplotlib.pyplot as plt
import  scipy.io#导入用来读取.mat 文件
import random
#读取数据
data='data/ex7data1.mat'
mat = scipy.io.loadmat(data)
cols = mat['X']
m=cols.shape[0]#数据项大小
iterations = 1#初始化随机选取数据簇中心的K值次数
C_i=np.zeros((m,1))#初始化ci，ci的第i个值分别代表对应xi所属的第几类
K=3#初始化簇类的个数
k_cluster_vals=np.zeros((K,cols.shape[1]))#初始化第k个簇中心的点的值

#初步观察数据
def plot_init_data():
    plt.figure(figsize=(8,6))
    plt.plot(cols[:,0],cols[:,1],'bo',label='init data points')
    plt.grid(True)
    plt.title('Training data plot')
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.legend()
#lot_init_data()
#C_i
#k_cluster_vals
#构建K-means算法
list_v=[]
list_k=[]
flist=np.array([])
def K_means(K,X):
    closest_k=1#将该点所属簇初始为1，计算距离
    closest_val = 1000000#初始最小距离为0
    for i in range(iterations):#随机初始化簇中心点选取点的值，总共随机选取iterations次
        #随机生成K个簇中心值
        k_cluster_vals=np.array(random.sample(list(cols),K)).reshape((K,cols.shape[1]))
        print(k_cluster_vals)
        #然后分别计算数据集中每一个点与簇中心点的距离，选取最近的簇中心点，并且归到该点中，即设置C-i的值
        for j in range(X.shape[0]):
            for o in range(k_cluster_vals.shape[0]):
                val=0
                for p in range(k_cluster_vals.shape[1]):
                    val= val+(k_cluster_vals[o][p]-X[j][p])**2
                list_v.append(val)
                if(closest_val>val):
                    closet_val=val
    #根据运算得到C-i,使用方法np.argmin(array,axis=0/1),当axis为1时时按每一行求出最小值的索引，为0时则为列
    #print(list_v)
    #print(np.array(list_v).reshape((X.shape[0],K)))
    C_i=np.argmin(np.array(list_v).reshape((X.shape[0],K)),axis=1).reshape((X.shape[0],1))#得到的ci中0代表第一类，1代表第二类。。。
    #print(C_i)
    #得到C_i之后然后根据新的分类点更新不同簇的中心点的值，为该簇类所有点的均值
    X=np.hstack((X,C_i))
    for i in range(K):
        #更新第i类的中心点的值
        mask = X[:,2]==i
        
K_means(K,cols)
#print(cols.shape)



[[2.53446019 1.21562081]
 [5.69797866 2.94977132]
 [5.87655237 3.21661109]]


In [121]:
(6.00506534-5.19590823)**2+(2.72784171-3.06972937)**2

0.771622400723828

In [110]:
(6.00506534-2.98340757)**2+(2.72784171-4.84046406)**2

13.593588872720899